# G-Eval

* Naturalness(1-5) : 응답이 사람의 자연스러운 발화와 같은지 판단
* coherence (1-5) : 이전 대화와 이어지는지 (연속적인지) 판단
* Engagingness (1-3) : 응답이 흥미로운지 지루한지 판단
* Groundedness (1-3) : 응답이 사실을 포함하는지

----

논문 원문에서는 Engagingness 지표에 대한 scale & definition만 나와있으므로 나머지는 직접 설계해야 함


## GPT api call


In [1]:
from openai import OpenAI
import os
import dotenv

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ["OPENAI_API_KEY"],
)

## 생성된 CoT 불러오기

In [2]:
coherence_prompt = open('cot/coherence_KOR3.txt').read()
engagingness_prompt = open('cot/engagingness_KOR3.txt').read()
naturalness_prompt = open('cot/naturalness_KOR3.txt').read()
groundedness_prompt = open('cot/groundedness_KOR3.txt').read()

In [3]:
eval_metric = {'coherence' : coherence_prompt, 'engagingness' : engagingness_prompt, 'naturalness' : naturalness_prompt, 'groundedness' : groundedness_prompt}

## 평가 함수 정의 (GPT-4)

In [4]:
# 저장 경로 설정
RESULT_PATH = './results'

# 모델이 생성하지 못한 응답 예외 처리
skip_str_list = ['', '\n']

In [6]:
from tqdm import tqdm
import time
from collections import Counter
import json

def evaluate(data, info):
    '''
    data : 평가할 데이터
    info : 어떤 모델을 평가하는지 (json file name에 쓰임)
    '''
    new_json = []
    ignore = 0

    for instance in tqdm(data):
        user_persona = '. '.join(instance['user_persona'])
        bot_persona = ''.join(instance['bot_persona'])
        prev_dialog = '\n'.join(instance['prev_dialog'])
        bot_response = instance['response']

        # 불용어 처리 
        if bot_response in skip_str_list or len(bot_response) < 2:
            ignore += 1
            print(f'## {bot_response} has been ignored.\n## total ignored : {ignore}')
            continue

        # unieval 참조 - 4개의 프롬프트에 대해 실험 진행
        for prompt_info, prompt in eval_metric.items():
            prompt_cur = prompt.replace('{{user_persona}}', user_persona).replace('{{bot_persona}}', bot_persona).replace('{{prev_dialog}}', prev_dialog).replace('{{response}}', bot_response)

            while True:
                try:
                    response = client.chat.completions.create(
                                    model="gpt-4",
                                    messages=[{"role": "system", "content": prompt_cur}],
                                    temperature=1,
                                    max_tokens=1, # 정수 형태 추출
                                    frequency_penalty=0,
                                    presence_penalty=0,
                                    stop=None,
                                    n=10 # 10번 추출
                    )

                    time.sleep(0.5)

                    
                    all_responses = [response.choices[i].message.content for i in
                                    range(len(response.choices))]
                    
                    # 가중합 내주기
                    # 전부 숫자 형태로 변환
                    all_responses = [int(r) for r in all_responses]
                    # print('## all_reponses : ', all_responses)

                    # 몇 번 나왔는지 체크
                    counter = Counter(all_responses)
                    # (1*5 + 2*4 + 5*1) / 10
                    score = sum([key * value for key, value in counter.items()]) / len(all_responses)
                    print('## all_responses : ', all_responses)
                    print(f'## {prompt_info} score : ', score)
                    
                    # 평가 체크
                    instance[prompt_info] = score
                    break

                except Exception as e:
                    print(e)
                    if ("limit" in str(e)):
                        time.sleep(2)
                    else:
                        ignore += 1
                        print('ignored', ignore)
                        break

        new_json.append(instance)
            

    print(f'## Evaluated {len(data)-ignore}/{len(data)} row! : ', )
    with open(f"{RESULT_PATH}/{info}.json", 'w', encoding="UTF-8") as f:
        json.dump(new_json, f, indent=4, ensure_ascii=False)


## 모델이 발화 예측 완료한 파일 불러오기

In [7]:
import json
# G-EVAL
solar_vl2_1 = json.load(open("./korean_msc/preprocessing/GEVAL_solar_vl2_persona_1.json"))
solar_vl2_no_persona_1 = json.load(open("./korean_msc/preprocessing/GEVAL_No_Persona_solar_vl2_1.json"))

solar_vl2_2 = json.load(open("./korean_msc/preprocessing/GEVAL_solar_vl2_persona_2.json"))
solar_vl2_no_persona_2 = json.load(open("./korean_msc/preprocessing/GEVAL_No_Persona_solar_vl2_2.json"))

solar_vl2_3 = json.load(open("./korean_msc/preprocessing/GEVAL_solar_vl2_persona_1.json"))
solar_vl2_no_persona_3 = json.load(open("./korean_msc/preprocessing/GEVAL_No_Persona_solar_vl2_3.json"))

In [8]:
skipped = 0
skip_str_list = ['', '\n']

for i in solar_vl2_2:
    if i['response'] in skip_str_list or len(i['response']) < 2:
        skipped += 1

print('## total skipped in SOLAR: ', skipped)

## total skipped in SOLAR:  0


In [9]:
skipped = 0
skip_str_list = ['', '\n']

for i in solar_vl2_no_persona_2:
    if i['response'] in skip_str_list or len(i['response']) < 2:
        print(i['response'])
        skipped += 1

print('## total skipped in SOLAR - No persona: ', skipped)

## total skipped in SOLAR - No persona:  0


In [10]:
evaluate(solar_vl2_2, 'GEVAL_solar_vl2_2')

  0%|          | 0/54 [00:00<?, ?it/s]

## all_reponses :  [5, 4, 4, 5, 5, 3, 4, 5, 5, 5]
## coherence score :  [5, 4, 4, 5, 5, 3, 4, 5, 5, 5]
## all_reponses :  [4, 3, 3, 3, 3, 3, 3, 3, 3, 3]
## engagingness score :  [4, 3, 3, 3, 3, 3, 3, 3, 3, 3]
## all_reponses :  [5, 4, 4, 5, 4, 4, 4, 5, 4, 4]
## naturalness score :  [5, 4, 4, 5, 4, 4, 4, 5, 4, 4]


  2%|▏         | 1/54 [00:05<04:47,  5.43s/it]

## all_reponses :  [4, 5, 3, 4, 4, 3, 5, 4, 3, 3]
## groundedness score :  [4, 5, 3, 4, 4, 3, 5, 4, 3, 3]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 4, 4]
## coherence score :  [5, 5, 5, 5, 5, 5, 5, 5, 4, 4]
## all_reponses :  [3, 5, 3, 5, 3, 5, 4, 5, 4, 3]
## engagingness score :  [3, 5, 3, 5, 3, 5, 4, 5, 4, 3]
## all_reponses :  [3, 5, 4, 4, 4, 4, 5, 4, 5, 4]
## naturalness score :  [3, 5, 4, 4, 4, 4, 5, 4, 5, 4]


  4%|▎         | 2/54 [00:10<04:41,  5.41s/it]

...

## all_reponses :  [2, 3, 1, 2, 1, 2, 1, 4, 3, 1]
## groundedness score :  [2, 3, 1, 2, 1, 2, 1, 4, 3, 1]
## all_reponses :  [5, 5, 3, 4, 4, 3, 4, 5, 3, 5]
## coherence score :  [5, 5, 3, 4, 4, 3, 4, 5, 3, 5]
## all_reponses :  [2, 4, 4, 3, 3, 4, 4, 3, 3, 3]
## engagingness score :  [2, 4, 4, 3, 3, 4, 4, 3, 3, 3]
## all_reponses :  [5, 3, 4, 4, 3, 4, 4, 4, 4, 5]
## naturalness score :  [5, 3, 4, 4, 3, 4, 4, 4, 4, 5]


 98%|█████████▊| 53/54 [05:09<00:05,  5.52s/it]

## all_reponses :  [4, 5, 2, 4, 2, 3, 3, 2, 4, 3]
## groundedness score :  [4, 5, 2, 4, 2, 3, 3, 2, 4, 3]
## all_reponses :  [4, 3, 1, 4, 5, 4, 2, 4, 4, 3]
## coherence score :  [4, 3, 1, 4, 5, 4, 2, 4, 4, 3]
## all_reponses :  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1]
## engagingness score :  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1]
## all_reponses :  [2, 3, 2, 4, 4, 3, 5, 2, 5, 5]
## naturalness score :  [2, 3, 2, 4, 4, 3, 5, 2, 5, 5]


100%|██████████| 54/54 [05:14<00:00,  5.83s/it]

## all_reponses :  [3, 2, 2, 3, 5, 5, 4, 4, 4, 4]
## groundedness score :  [3, 2, 2, 3, 5, 5, 4, 4, 4, 4]
## Evaluated 54/54 row! : 


In [11]:
evaluate(solar_vl2_no_persona_2, 'GEVAL_solar_vl2_no_persona_2')

  0%|          | 0/54 [00:00<?, ?it/s]

## all_reponses :  [5, 5, 4, 5, 5, 5, 5, 5, 5, 5]
## coherence score :  [5, 5, 4, 5, 5, 5, 5, 5, 5, 5]
## all_reponses :  [3, 3, 2, 3, 4, 3, 3, 3, 2, 3]
## engagingness score :  [3, 3, 2, 3, 4, 3, 3, 3, 2, 3]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## naturalness score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


  2%|▏         | 1/54 [00:05<04:46,  5.40s/it]

## all_reponses :  [5, 4, 4, 5, 5, 4, 3, 5, 4, 5]
## groundedness score :  [5, 4, 4, 5, 5, 4, 3, 5, 4, 5]
## all_reponses :  [4, 5, 5, 5, 5, 5, 5, 4, 5, 5]
## coherence score :  [4, 5, 5, 5, 5, 5, 5, 4, 5, 5]
## all_reponses :  [3, 4, 5, 4, 5, 4, 3, 4, 3, 4]
## engagingness score :  [3, 4, 5, 4, 5, 4, 3, 4, 3, 4]
## all_reponses :  [3, 5, 3, 5, 4, 5, 4, 4, 4, 4]
## naturalness score :  [3, 5, 3, 5, 4, 5, 4, 4, 4, 4]


  4%|▎         | 2/54 [00:10<04:43,  5.45s/it]

...

## all_reponses :  [4, 3, 4, 5, 4, 4, 4, 3, 4, 2]
## groundedness score :  [4, 3, 4, 5, 4, 4, 4, 3, 4, 2]
## all_reponses :  [3, 3, 3, 4, 4, 4, 3, 2, 5, 5]
## coherence score :  [3, 3, 3, 4, 4, 4, 3, 2, 5, 5]
## all_reponses :  [4, 2, 3, 3, 3, 3, 4, 4, 3, 3]
## engagingness score :  [4, 2, 3, 3, 3, 3, 4, 4, 3, 3]
## all_reponses :  [4, 3, 4, 4, 4, 3, 4, 3, 4, 3]
## naturalness score :  [4, 3, 4, 4, 4, 3, 4, 3, 4, 3]


 98%|█████████▊| 53/54 [04:42<00:05,  5.50s/it]

## all_reponses :  [2, 3, 3, 2, 2, 3, 3, 2, 4, 3]
## groundedness score :  [2, 3, 3, 2, 2, 3, 3, 2, 4, 3]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## coherence score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## all_reponses :  [2, 2, 3, 1, 4, 1, 3, 2, 3, 3]
## engagingness score :  [2, 2, 3, 1, 4, 1, 3, 2, 3, 3]
## all_reponses :  [4, 4, 5, 4, 4, 5, 5, 5, 5, 5]
## naturalness score :  [4, 4, 5, 4, 4, 5, 5, 5, 5, 5]


100%|██████████| 54/54 [04:48<00:00,  5.34s/it]

## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 4, 5, 5]
## groundedness score :  [5, 5, 5, 5, 5, 5, 5, 4, 5, 5]
## Evaluated 54/54 row! : 


In [12]:
evaluate(solar_vl2_3, 'GEVAL_solar_vl2_3')

  0%|          | 0/54 [00:00<?, ?it/s]

## all_reponses :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
## coherence score :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
## all_reponses :  [1, 1, 2, 1, 1, 1, 1, 1, 1, 1]
## engagingness score :  [1, 1, 2, 1, 1, 1, 1, 1, 1, 1]
## all_reponses :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
## naturalness score :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


  2%|▏         | 1/54 [00:05<05:01,  5.69s/it]

## all_reponses :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
## groundedness score :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## coherence score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## all_reponses :  [3, 3, 3, 3, 4, 3, 3, 3, 3, 4]
## engagingness score :  [3, 3, 3, 3, 4, 3, 3, 3, 3, 4]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## naturalness score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


  4%|▎         | 2/54 [00:11<04:48,  5.55s/it]

...

## all_reponses :  [4, 2, 4, 3, 4, 3, 3, 1, 3, 3]
## groundedness score :  [4, 2, 4, 3, 4, 3, 3, 1, 3, 3]
## all_reponses :  [3, 2, 3, 3, 4, 2, 1, 4, 4, 2]
## coherence score :  [3, 2, 3, 3, 4, 2, 1, 4, 4, 2]
## all_reponses :  [3, 3, 3, 3, 4, 3, 3, 3, 2, 3]
## engagingness score :  [3, 3, 3, 3, 4, 3, 3, 3, 2, 3]
## all_reponses :  [3, 3, 4, 2, 4, 3, 3, 4, 3, 2]
## naturalness score :  [3, 3, 4, 2, 4, 3, 3, 4, 3, 2]


 98%|█████████▊| 53/54 [04:51<00:05,  5.42s/it]

## all_reponses :  [2, 2, 2, 1, 3, 3, 3, 3, 2, 3]
## groundedness score :  [2, 2, 2, 1, 3, 3, 3, 3, 2, 3]
## all_reponses :  [3, 3, 3, 2, 3, 1, 2, 1, 2, 4]
## coherence score :  [3, 3, 3, 2, 3, 1, 2, 1, 2, 4]
## all_reponses :  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]
## engagingness score :  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]
## all_reponses :  [1, 1, 3, 3, 2, 4, 2, 2, 1, 2]
## naturalness score :  [1, 1, 3, 3, 2, 4, 2, 2, 1, 2]


100%|██████████| 54/54 [04:56<00:00,  5.50s/it]

## all_reponses :  [2, 4, 4, 4, 2, 4, 3, 4, 3, 4]
## groundedness score :  [2, 4, 4, 4, 2, 4, 3, 4, 3, 4]
## Evaluated 54/54 row! : 


In [13]:
evaluate(solar_vl2_no_persona_3, 'GEVAL_solar_vl2_no_persona_3')

  0%|          | 0/54 [00:00<?, ?it/s]

## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## coherence score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## all_reponses :  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
## engagingness score :  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## naturalness score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


  2%|▏         | 1/54 [00:05<04:47,  5.42s/it]

## all_reponses :  [5, 5, 3, 5, 5, 3, 4, 5, 5, 5]
## groundedness score :  [5, 5, 3, 5, 5, 3, 4, 5, 5, 5]
## all_reponses :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## coherence score :  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
## all_reponses :  [5, 5, 5, 4, 4, 3, 4, 4, 4, 4]
## engagingness score :  [5, 5, 5, 4, 4, 3, 4, 4, 4, 4]
## all_reponses :  [5, 5, 5, 5, 5, 5, 4, 5, 4, 5]
## naturalness score :  [5, 5, 5, 5, 5, 5, 4, 5, 4, 5]


  4%|▎         | 2/54 [00:10<04:35,  5.30s/it]

...

## all_reponses :  [3, 3, 5, 4, 3, 3, 2, 2, 2, 2]
## groundedness score :  [3, 3, 5, 4, 3, 3, 2, 2, 2, 2]
## all_reponses :  [4, 2, 4, 5, 4, 3, 4, 5, 4, 1]
## coherence score :  [4, 2, 4, 5, 4, 3, 4, 5, 4, 1]
## all_reponses :  [2, 3, 2, 2, 3, 2, 3, 3, 2, 2]
## engagingness score :  [2, 3, 2, 2, 3, 2, 3, 3, 2, 2]
## all_reponses :  [4, 3, 2, 3, 3, 2, 4, 4, 4, 3]
## naturalness score :  [4, 3, 2, 3, 3, 2, 4, 4, 4, 3]


 98%|█████████▊| 53/54 [04:40<00:05,  5.30s/it]

## all_reponses :  [4, 4, 3, 4, 4, 4, 3, 4, 3, 3]
## groundedness score :  [4, 4, 3, 4, 4, 4, 3, 4, 3, 3]
## all_reponses :  [5, 4, 5, 4, 3, 4, 4, 5, 5, 3]
## coherence score :  [5, 4, 5, 4, 3, 4, 4, 5, 5, 3]
## all_reponses :  [2, 3, 3, 3, 3, 3, 3, 2, 3, 2]
## engagingness score :  [2, 3, 3, 3, 3, 3, 3, 2, 3, 2]
## all_reponses :  [5, 4, 3, 4, 4, 4, 4, 4, 4, 5]
## naturalness score :  [5, 4, 3, 4, 4, 4, 4, 4, 4, 5]


100%|██████████| 54/54 [04:45<00:00,  5.29s/it]

## all_reponses :  [3, 3, 4, 3, 4, 3, 3, 2, 4, 4]
## groundedness score :  [3, 3, 4, 3, 4, 3, 3, 2, 4, 4]
## Evaluated 54/54 row! : 


In [ ]:
evaluate(solar_vl2_3, 'GEVAL_solar_vl2_3')